In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# 1. Initialiser SparkSession
spark = SparkSession.builder \
    .appName("Data Fusion - Weather and Accidents") \
    .getOrCreate()

24/12/10 11:54:50 WARN Utils: Your hostname, MacBook-Air-Madina.local resolves to a loopback address: 127.0.0.1; using 10.188.13.32 instead (on interface en0)
24/12/10 11:54:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 11:54:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/10 11:54:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# 2. Charger les données météo (JSON)
weather_df = spark.read.json("/Users/madina/Desktop/datalake/weather_data.json")

In [12]:
weather_df = spark.read.option("multiLine", True).json("/Users/madina/Desktop/datalake/weather_data.json", mode="PERMISSIVE")


In [13]:
# Afficher les lignes corrompues
corrupted = weather_df.filter(weather_df["_corrupt_record"].isNotNull())
corrupted.show(truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `_corrupt_record` cannot be resolved. Did you mean one of the following? [`AccidentIndex`, `Date`, `DayLengthHours`, `MaxTemperature`, `MaxWindSpeed`, `MinTemperature`, `SolarRadiation`, `TotalPrecipitation`].

In [7]:
# 3. Charger les données d'accidents (CSV)
accidents_df = spark.read.csv("/Users/madina/Desktop/datalake/csv-accident/road clean.csv", header=True, inferSchema=True)


In [14]:
# Fusionner les deux DataFrames sur la colonne AccidentIndex
merged_df = accidents_df.join(weather_df, on="AccidentIndex", how="inner")

In [16]:
# Afficher les données fusionnées
merged_df.show(5)

24/12/10 14:28:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+-------------+-----------+----------------+---------------+-----------------+---------+----------------+--------------------------+---------+--------------------+------------------+-------------------+-----------------------+---------+-----------+-------------------+-------------------+------------+----------+------------------+--------------+------------+--------------+--------------+------------------+
|AccidentIndex|Accident Date|Day_of_Week|Junction_Control|Junction_Detail|Accident_Severity| Latitude|Light_Conditions|Local_Authority_(District)|Longitude|Number_of_Casualties|Number_of_Vehicles|       Police_Force|Road_Surface_Conditions|Road_Type|Speed_limit|               Time|Urban_or_Rural_Area|Vehicle_Type|      Date|    DayLengthHours|MaxTemperature|MaxWindSpeed|MinTemperature|SolarRadiation|TotalPrecipitation|
+-------------+-------------+-----------+----------------+---------------+-----------------+---------+----------------+--------------------------+--------

In [ ]:
# Sauvegarde du résultat
output_path = "/Users/madina/Desktop/datalake/Merged_Accidents_Weather_Data.csv"
merged_df.coalesce(1).write.option("header", True).csv(output_path)
print(f"Les données fusionnées ont été sauvegardées dans le fichier : {output_path}")
